In [1]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import  LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

load_dotenv()  
URI = os.environ["NEO4J_INSTANCE01_URI"]
USER = os.environ["NEO4J_INSTANCE01_USER"]
PWD = os.environ["NEO4J_INSTANCE01_KEY"]

graph = Neo4jGraph(url=URI, username=USER, password=os.environ["NEO4J_INSTANCE01_KEY"])



In [3]:
neo4j_search_chain = GraphCypherQAChain.from_llm(
     graph=graph,
     cypher_llm=ChatOpenAI(temperature=0, model="gpt-4", openai_api_key= os.environ["OPENAI_API_KEY"]),
     qa_llm=ChatOpenAI(temperature=0, model="gpt-4", openai_api_key= os.environ["OPENAI_API_KEY"]),
     verbose=True,
     return_intermediate_steps=True,
     top_k=100
)

In [ ]:
tools = [
    Tool(
        name = "Search in Northwind online shop graph database",
        func=neo4j_search_chain.run,
        description=f"""This function is useful to retreive information from the Northwind online shop graph database. 
The schema of this database is:
{graph.get_schema}"""
    )]

In [ ]:
# Set up the base template
template = """
You will be asked a question about the Northwind online shop. 
You have to answer to question using the following tool
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: The input question you must answer
Thought: you should always think about what to do
Action: the action to take is always to retreive the right information from the Northwind online shop.
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Don't forget you can repeat multiple time Thought/Action/Action Input/Observation until you have all the information you need to answer the question
Question: {input}
{agent_scratchpad}"""

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [ ]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

llm = ChatOpenAI(temperature=0, model="gpt-4-0613")

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)


In [ ]:
agent_executor.run("How many people live in canada as of 2023?")